# Part 3 - Run simulations with stagnation points / velocities from experiment

### First load some references

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

### Init Database etc.

In [ ]:
string ProjectName4Correlation = "SFB1194_K65_Part1";
string ProjectName = "SFB1194_K65_Part3";

In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName4Correlation);
static var myDb4Correlation = BoSSSshell.OpenDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase.Path);
Console.WriteLine("Initializing : " + myDb4Correlation.Path); // we need to init the static var! otherwise we end up with a reference to the wrong database!

Project name is set to 'SFB1194_K65_Part1'.
Opening existing database '\\hpccluster\hpccluster-scratch\rieckmann\SFB1194_K65_Part1'.
Initializing : \\hpccluster\hpccluster-scratch\rieckmann\SFB1194_K65_Part1


In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);
static var myDb = BoSSSshell.OpenDatabase(BoSSSshell.WorkflowMgm.DefaultDatabase.Path);
Console.WriteLine("Initializing : " + myDb.Path);

Project name is set to 'SFB1194_K65_Part3'.
Opening existing database '\\hpccluster\hpccluster-scratch\rieckmann\SFB1194_K65_Part3'.
Initializing : \\hpccluster\hpccluster-scratch\rieckmann\SFB1194_K65_Part3


In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

### Construct parameterized velocity splines

In [ ]:
double H0 = 1e-5;
double V0 = 1.0;
double P0 = 1e6;

In [ ]:
var sessions = myDb4Correlation.Sessions.Where(s => s.ProjectName == "SFB1194_K65_Part1").ToArray();
var sV = sessions.Single(s => Convert.ToDouble(s.KeysAndQueries["id:delta"]).ApproxEqual(H0) &  Convert.ToDouble(s.KeysAndQueries["id:P_Diff"]).ApproxEqual(0.0) & Convert.ToDouble(s.KeysAndQueries["id:V_Wall"]).ApproxEqual(V0));
var sP = sessions.Single(s => Convert.ToDouble(s.KeysAndQueries["id:delta"]).ApproxEqual(H0) &  Convert.ToDouble(s.KeysAndQueries["id:P_Diff"]).ApproxEqual(P0) & Convert.ToDouble(s.KeysAndQueries["id:V_Wall"]).ApproxEqual(0.0));

Function to create the velocity profile along the symmetry axis

In [ ]:
using MathNet.Numerics.Interpolation;

In [ ]:
static public CubicSpline ConstructVelocitySpline(ISessionInfo si) {
    DGField VelocityX = si.Timesteps.Last().Fields.Single(f => f.Identification == "VelocityX");
    var grd = (GridData)VelocityX.GridDat;
    EdgeMask em = new EdgeMask(grd, X => Math.Abs(X[1]) < 1e-12); // symmetry axis

    var Spline = Postprocessing.SplineOnEdge(em, VelocityX, 0, out double lB, out double uB);
    return Spline;
}

Evaluate the profile for both simulations and superpose

In [ ]:
var SplineV = ConstructVelocitySpline(sV);
var SplineP = ConstructVelocitySpline(sP);

In [ ]:
Func<double, double, double, double, double> VelocityFunc = (x, h, v, p) => v/V0 * SplineV.Interpolate(x * Math.Sqrt(H0) / Math.Sqrt(h)) + (p * Math.Pow(h,1.5))/(P0 * Math.Pow(H0,1.5)) * SplineP.Interpolate(x * Math.Sqrt(H0) / Math.Sqrt(h));

Set Velocity to be zero at a certain x_stag and calculate the necessary pressure difference

In [ ]:
Func<double, double, double, double> PressureFunc = (x_stag, h, v) => -v/V0 * SplineV.Interpolate(x_stag * Math.Sqrt(H0) / Math.Sqrt(h)) * (P0 * Math.Pow(H0,1.5)) / (Math.Pow(h,1.5) * SplineP.Interpolate(x_stag * Math.Sqrt(H0) / Math.Sqrt(h)));

### Setup Simulations

Experimental Data - See PrintingNip_Experiment.pptx

In [ ]:
// List<Tuple<double, double, double>> Setup = new List<Tuple<double, double, double>>(); // <Raster,V,x_stag>
// Setup.Add(Tuple.Create(60.0, 10.0 / 60.0, 0.0092721));
// Setup.Add(Tuple.Create(60.0, 20.0 / 60.0, 0.0103293));
// Setup.Add(Tuple.Create(60.0, 30.0 / 60.0, 0.0106930));
// Setup.Add(Tuple.Create(60.0, 45.0 / 60.0, 0.0116885));

// Setup.Add(Tuple.Create(70.0, 10.0 / 60.0, 0.0090436));
// Setup.Add(Tuple.Create(70.0, 20.0 / 60.0, 0.0099132));
// Setup.Add(Tuple.Create(70.0, 30.0 / 60.0, 0.0105802));
// Setup.Add(Tuple.Create(70.0, 45.0 / 60.0, 0.0108676));
// Setup.Add(Tuple.Create(70.0, 60.0 / 60.0, 0.0105994));
// Setup.Add(Tuple.Create(70.0, 75.0 / 60.0, 0.0106655));

// Setup.Add(Tuple.Create(80.0, 10.0 / 60.0, 0.0085139));
// Setup.Add(Tuple.Create(80.0, 20.0 / 60.0, 0.0098213));
// Setup.Add(Tuple.Create(80.0, 30.0 / 60.0, 0.0098796));
// Setup.Add(Tuple.Create(80.0, 45.0 / 60.0, 0.0102255));
// Setup.Add(Tuple.Create(80.0, 60.0 / 60.0, 0.0105994));
// Setup.Add(Tuple.Create(80.0, 75.0 / 60.0, 0.0104337));

// Setup.Add(Tuple.Create(100.0, 10.0 / 60.0, 0.0081541));
// Setup.Add(Tuple.Create(100.0, 20.0 / 60.0, 0.0085932));
// Setup.Add(Tuple.Create(100.0, 30.0 / 60.0, 0.0086691));
// Setup.Add(Tuple.Create(100.0, 45.0 / 60.0, 0.0085610)); // ??
// Setup.Add(Tuple.Create(100.0, 60.0 / 60.0, 0.0092080));
// Setup.Add(Tuple.Create(100.0, 75.0 / 60.0, 0.0097090));

In [ ]:
double[] deltaS = new double[] {
    //0.001, // millimeters
    //0.0005, 
    //0.0001, 
    //0.00005, 
    0.00001, 
    0.000005, 
    0.000001 }; // micro-meter

// always same
int Res = 10;
int DGdegree = 5;
double R = 0.1;
double x_stag = 0.01; // always at 10 mm

// set grid to be saved in database
GridFactory.myDb = myDb;

In [ ]:
string[] lines = File.ReadAllLines("./ExperimentalResults.txt");
List<Tuple<double, double, double>> Setup = new List<Tuple<double, double, double>>(); // <Raster,V,x_stag>

double e = 0.0395; // viscosity
double s = 0.027; // surface tension

for(int i = 1; i < lines.Length; i++){
    string line = lines[i];
    string[] values = line.Split('\t');
    double raster = Convert.ToDouble(values[0]);
    double velocity = Convert.ToDouble(values[1]);
    double finger = Convert.ToDouble(values[2])*100; // convert to 1/m

    double Ca = e * velocity / s;
    double h_m = Math.Sqrt(Ca / (finger * finger));
    double x_m = Math.Sqrt(h_m*R); // cylinder/cylinder, for cylinder/plate sqrt(2*h*R)
    double x_stag = x_m;
    
    Setup.Add(Tuple.Create(raster, velocity, x_stag));
}

In [ ]:
Setup

index,Item1,Item2,Item3
0,60,0.1666667,0.00666894324961969
1,60,0.3333333,0.007348809514636953
2,60,0.5,0.007544082935209117
3,60,0.75,0.008161259644642382
4,60,1,0.008430784373634197
5,60,1.25,0.010778377157023017
6,70,0.1666667,0.006396007895588078
7,70,0.3333333,0.0071020858851714545
8,70,0.5,0.007318451188571249
9,70,0.75,0.007648884957317737


In [ ]:
var controls = new List<XNSE_Control>();

foreach(double delta in deltaS) {
   foreach(Tuple<double, double, double> R_V_X in Setup) {

      double V_wall = R_V_X.Item2;
      double p = PressureFunc(R_V_X.Item3, delta, V_wall);

      var C = new XNSE_Control();
      C.SetDGdegree(DGdegree);
      C.SetGrid(GridFactory.GenerateGrid(Res,delta, R));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("delta", delta));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Radius", R));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("V_Wall", V_wall));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("P_Diff", p));
      C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Raster", R_V_X.Item1));



      C.AddBoundaryValue("wall_walze", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
      C.AddBoundaryValue("wall_walze", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
      C.AddBoundaryValue("wall_substrat", "VelocityX", BoundaryValueFactory.Get_VelX(delta, V_wall, R));
      C.AddBoundaryValue("wall_substrat", "VelocityY", BoundaryValueFactory.Get_VelY(delta, V_wall, R));
      C.AddBoundaryValue("pressure_outlet_in", "Pressure", $"X => {p}", false);
      C.AddBoundaryValue("pressure_outlet_out", "Pressure", $"X => {-p}", false);


      C.TimesteppingMode = AppControl._TimesteppingMode.Steady;

      C.PhysicalParameters.rho_A             = 1026.4;
      C.PhysicalParameters.mu_A              = 0.0395;
      C.PhysicalParameters.IncludeConvection = false;
      C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
      
      C.SessionName = "J" + (20*Res*Res) + "_delta" + delta + "_V" + V_wall + "_P" + p + "_RL" + R_V_X.Item1;     
      controls.Add(C);
   }
}

Grid Edge Tags changed.
Grid Edge Tags changed.
Grid Edge Tags changed.


In [ ]:
Console.WriteLine(controls.Count);

72


### Run Simulations

Workaround so we need to deploy the rather large executables only once!

In [ ]:
string mngdir = "SFB1194K65_Part3_" + DateTime.Now.ToString("MMMdd_HHmmss") + "_managed";
DirectoryInfo ManagedOverride = new DirectoryInfo(Path.Combine(myBatch.DeploymentBaseDirectory, mngdir));
ManagedOverride.Create();
typeof(XNSE<XNSE_Control>).Assembly.DeployAt(ManagedOverride);
string RelManagedPath = "../" + mngdir + "/" + Path.GetFileName(typeof(XNSE<XNSE_Control>).Assembly.Location);

In [ ]:
string ntvdir = "SFB1194K65_Part3_" + DateTime.Now.ToString("MMMdd_HHmmss") + "_amd64";
DirectoryInfo NativeOverride = new DirectoryInfo(Path.Combine(myBatch.DeploymentBaseDirectory, ntvdir));
NativeOverride.Create();
MetaJobMgrIO.CopyDirectoryRec(ilPSP.Environment.NativeLibraryDir, NativeOverride.FullName, null);
myBatch.DeployRuntime = false; // we did this manually!

In [ ]:
foreach(var c in controls) {
    Job j = c.CreateJob();
    j.EnvironmentVars.Add(BoSSS.Foundation.IO.Utils.BOSSS_NATIVE_OVERRIDE, NativeOverride.FullName);
    j.EntryAssemblyRedirection = RelManagedPath;
    j.Activate(myBatch);
}

Deploying job J2000_delta1E-05_V0.1666667_P6430265.902163791_RL60 ... 
Deploying executables and additional files ...
Skipping assembly file copy, since 'EntryAssemblyRedirection' = ../SFB1194K65_Part3_May04_100225_managed/XNSE_Solver.dll
Deployment directory: \\hpccluster\hpccluster-scratch\rieckmann\binaries\SFB1194_K65_Part3-XNSE_Solver2022May04_100502
copied 0 files.
   written file: control.obj
deployment finished.

Deploying job J2000_delta1E-05_V0.3333333_P15814166.37990894_RL60 ... 
Deploying executables and additional files ...
Skipping assembly file copy, since 'EntryAssemblyRedirection' = ../SFB1194K65_Part3_May04_100225_managed/XNSE_Solver.dll
Deployment directory: \\hpccluster\hpccluster-scratch\rieckmann\binaries\SFB1194_K65_Part3-XNSE_Solver2022May04_100627
copied 0 files.
   written file: control.obj
deployment finished.

Deploying job J2000_delta1E-05_V0.5_P25073051.433433536_RL60 ... 
Deploying executables and additional files ...
Skipping assembly file copy, since 'E

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(18000);